# Hotel Rating Prediction
In this project we will inspect the question   
whether we can or can not predict the overall rating on a given hotel  
according to different parameters which we already know

## Data Acquisition 

At this step we are going to acquire the data we need from the website  
and create the DataFrame which will serve us throughout the project.

In [33]:
# First lets import the libraries we need :
import requests
import pandas as pd
import numpy as np
import scipy as sc
from bs4 import BeautifulSoup
import time
import os
import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# The next command will be responsible to display graphs in our notebook.
%matplotlib inline

### Crawling 

We will now fetch what we need from the website "Booking.com"

In [34]:
# creating the function to load a soup object 
def load_soup_object(link):
    response_link = requests.get(link ,headers=0)
    soup_object = BeautifulSoup(response_link.content, "html.parser")
    return soup_object

# Lets get the URL and parser it before we create our DataFrame
URL = "https://www.booking.com/he"
soup = load_soup_object(URL)

# Now we will create the lists that will serve as the columns for the DataFrame
Hotel_Name = []
Staff_Rating = []
Facilities_Rating = []
Cleanliness_Rating = []
Comfort_Rating = []
Value_For_Money_Rating = []
Location_Rating = []
Free_WiFi_Rating = []



# accessing the class that contains the list of regions - not only in israel 
# and creating a list of known cities to be extracted 
# counter will be used to save the index of the desired regions - then to specify the index withing the 'searches' list
searches = soup.find(class_="ia_section-container clearfix wcu-seo-regions-poi").find_all('a')
cities = ["אזור ירושלים","אזור המרכז","גליל","גליל עליון","כנרת","אזור תל אביב","דרום הארץ","ים המלח","נגב","צפון הארץ","רמת הגולן"]
names = []
indexes = []
href_list = []
counter = 0
# now, 'searches' is a list with all the regions,we want to loop through and extract only the ones we want
# we are first getting the name of each object in the list, and checking if it exists in our desired regeions list 'cities'
# if term is met, we are appending its index(integer) to the indexes list
# and extracting the corresponding link with the "href" tag and concatanating it to the original URL
for i in searches:
    name1 = i.get_text().replace("\n", "") 
    if name1 in cities:
        names.append(name1)
        indexes.append(counter)
        href = searches[counter].get("href").strip()
        URL2 = URL+href
        soup2 = load_soup_object(URL2)
        
        # now lets parse it so we can inspect and work with each desired page 
        # each region`s page has a specific "Watch all" link that needs to be accessed
        # therefore, we are accessing the class that contains the only object(link) we need
        # and concatanating the link to the original URL
        
        searches2 = soup2.find(class_="lp-bui-section bui-spacer--largest x2").find_all("a")
        print("counter is : ", counter)
        print("region is : ", name1)
        nexthref = URL + searches2[0].get("href").strip()
        print("full name : ", searches2[0].get_text())
        href_list.append(nexthref)
        print("\nhref is :\n",nexthref, "\n")
    counter+=1

# now after this for loop we have a list that contains all the links to the full page of each desired region

print(names)
print(indexes)
print(names[0])

print("hi")
print(Hotel_Name)
print("hi")
#print(href_list)


    
    

counter is :  0
region is :  גליל
full name :  

צפו בכל 948 המלונות ב-גליל



href is :
 https://www.booking.com/he/searchresults.he.html?label=gen173nr-1FCAEoggI46AdIDlgEaGqIAQGYAQ64AQrIAQXYAQHoAQH4AQ6IAgGoAgO4ApSgk5QGwAIB0gIkNzUyNjRlMGYtN2ZiNy00ZTUwLWEwMTUtNjhkZjlhZmE3ZmI22AIG4AIB;sid=9d4c6b59446fd092425278d3289c3575;region=3646& 

counter is :  1
region is :  ים המלח
full name :  

צפו בכל 60 המלונות ב-ים המלח



href is :
 https://www.booking.com/he/searchresults.he.html?label=gen173nr-1FCAEoggI46AdIDlgEaGqIAQGYAQ64AQrIAQXYAQHoAQH4AQ6IAgGoAgO4ApSgk5QGwAIB0gIkNzUyNjRlMGYtN2ZiNy00ZTUwLWEwMTUtNjhkZjlhZmE3ZmI22AIG4AIB;sid=2eb1985a287a12335d8b1587cffae1ed;region=3230& 

counter is :  2
region is :  צפון הארץ
full name :  

צפו בכל 1,509 המלונות ב-צפון הארץ



href is :
 https://www.booking.com/he/searchresults.he.html?label=gen173nr-1FCAEoggI46AdIDlgEaGqIAQGYAQ64AQrIAQXYAQHoAQH4AQ6IAgGoAgO4ApSgk5QGwAIB0gIkNzUyNjRlMGYtN2ZiNy00ZTUwLWEwMTUtNjhkZjlhZmE3ZmI22AIG4AIB&sid=fb72ca02286dfedadf5b

Now after we have a list of the correct and desired regions,  
we are going to access each region's page and loop through all other pages  
so we can extract the information we need regarding each item(hotel) in the page.

In [35]:
#seleniuim variables :
#PATH = "./chromedriver.exe"
#s=Service(PATH)
#driver = webdriver.Chrome(service=s)
# now that we have all the links, we need to parse each one with html.parser so we can work with it
print("hi")
counter2 = 0
for link in href_list: #i.e north, south, negev
    #driver.get(link)    
    city_soup = load_soup_object(link)
    number_of_pages = int(city_soup.find(class_="d493c719bc").find_all('li')[-1].get_text())
    hotels = city_soup.find(class_="d4924c9e74").find_all('h3') # find all links(names links) for all hotels in the page into a list.
    print("len hotels : ", len(hotels))
    # now we start extracting information for the dataframe. we start by taking the hotel names into the hotel name list
    for item in hotels: # looping through the h3 class for the hotels --- item == href of single hotel
        flag = 0
        print("hi2")
        print("hotel : ",item.a.find(class_="fcab3ed991 a23c043802").get_text())
        hotel_eng_name = ""
        #if the hotel name is in hebrew - we will now fetch the english name for it - straight from the URL
        # because the hotel name starts after the 33rd character in the URL
        item_for_name = item.a.get("href").strip()[33:]
        for s in item_for_name:
            if s == ".":
                break
            hotel_eng_name += s
        Hotel_Name.append(hotel_eng_name)
        print("hotel in english : ", hotel_eng_name)
        print("link is :",item.a.get("href").strip())
        hotel_soup = load_soup_object(item.a.get("href").strip())
        
        #now lets get inside the link of the current hotel - and fetch the information for our DataFrame
        hotel_props = hotel_soup.select(".v2_review-scores__body.v2_review-scores__body--compared_to_average li")
        hotel_props2 = hotel_props[14:]
        print("hotel props len is : ", len(hotel_props2))
        if (len(hotel_props2)) < 7:
            print("problem in hotel : ", Hotel_Name[counter2])
            print("hotel props : ", hotel_props2)
            flag = 1
        if flag == 1:
            Staff_Rating.append(0)
            Facilities_Rating.append(0)
            Cleanliness_Rating.append(0)
            Comfort_Rating.append(0)
            Value_For_Money_Rating.append(0)
            Location_Rating.append(0)
            Free_WiFi_Rating.append(0)
        else:
            Staff_Rating.append(hotel_props2[0].find(class_="c-score-bar__score").get_text())
            Facilities_Rating.append(hotel_props2[1].find(class_="c-score-bar__score").get_text())
            Cleanliness_Rating.append(hotel_props2[2].find(class_="c-score-bar__score").get_text())
            Comfort_Rating.append(hotel_props2[3].find(class_="c-score-bar__score").get_text())
            Value_For_Money_Rating.append(hotel_props2[4].find(class_="c-score-bar__score").get_text())
            Location_Rating.append(hotel_props2[5].find(class_="c-score-bar__score").get_text())
            Free_WiFi_Rating.append(hotel_props2[6].find(class_="c-score-bar__score").get_text())
        counter2 += 1
        if counter2 == 4:
            break

#     print("Staff_Rating",len(Staff_Rating),"\n",Staff_Rating)
#     print("Facilities_Rating",len(Facilities_Rating),"\n",Facilities_Rating)
#     print("Cleanliness_Rating",len(Cleanliness_Rating),"\n",Cleanliness_Rating)
#     print("Comfort_Rating",len(Comfort_Rating),"\n",Comfort_Rating)
#     print("Value_For_Money_Rating",len(Value_For_Money_Rating),"\n",Value_For_Money_Rating)
#     print("Location_Rating",len(Location_Rating),"\n",Location_Rating)
#     print("Free_WiFi_Rating",len(Free_WiFi_Rating),"\n",Free_WiFi_Rating)
#     print("Hotel_Names:", len(Hotel_Name),"\n",Hotel_Name)
#     print("first hotel is : ", Hotel_Name[0])
    
    dfdict = {"Hotel_Names":Hotel_Name,"Staff_Rating":Staff_Rating, "Facilities_Rating":Facilities_Rating, "Cleanliness_Rating":Cleanliness_Rating, "Comfort_Rating":Comfort_Rating, "Value_For_Money_Rating":Value_For_Money_Rating, "Location_Rating":Location_Rating, "Free_WiFi_Rating":Free_WiFi_Rating}
    #dfdict = {"Staff_Rating":Staff_Rating, "Facilities_Rating":Facilities_Rating, "Cleanliness_Rating":Cleanliness_Rating, "Comfort_Rating":Comfort_Rating, "Value_For_Money_Rating":Value_For_Money_Rating, "Location_Rating":Location_Rating, "Hotel_Names":Hotel_Name}

    df = pd.DataFrame(data=dfdict)
    df.style.set_properties(**{'text-align': 'left'})
    df.style
    print(df)
    break

        
    

hi
len hotels :  25
hi2
hotel :  ההר האציל
hotel in english :  hahar-jaatzil
link is : https://www.booking.com/hotel/il/hahar-jaatzil.he.html?label=gen173nr-1FCAEoggI46AdIDlgEaGqIAQGYAQ64AQrIAQXYAQHoAQH4AQ6IAgGoAgO4ApSgk5QGwAIB0gIkNzUyNjRlMGYtN2ZiNy00ZTUwLWEwMTUtNjhkZjlhZmE3ZmI22AIG4AIB&sid=9d4c6b59446fd092425278d3289c3575&aid=304142&ucfs=1&arphpl=1&dest_id=3646&dest_type=region&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=0eba4cb36597022f&srepoch=1652871271&from=searchresults#hotelTmpl
hotel props len is :  7
hi2
hotel :  Yarden Estate Boutique Hotel
hotel in english :  yarden-estate
link is : https://www.booking.com/hotel/il/yarden-estate.he.html?label=gen173nr-1FCAEoggI46AdIDlgEaGqIAQGYAQ64AQrIAQXYAQHoAQH4AQ6IAgGoAgO4ApSgk5QGwAIB0gIkNzUyNjRlMGYtN2ZiNy00ZTUwLWEwMTUtNjhkZjlhZmE3ZmI22AIG4AIB&sid=9d4c6b59446fd092425278d3289c3575&aid=304142&ucfs=1&arphpl=1&dest_id=3646&dest_type=region&group_adults=2&req_adults=2&no_room

In [36]:
df

,Hotel_Names,Staff_Rating,Facilities_Rating,Cleanliness_Rating,Comfort_Rating,Value_For_Money_Rating,Location_Rating,Free_WiFi_Rating
0,hahar-jaatzil,9.6,9.1,9.1,9.3,8.6,9.0,8.0
1,yarden-estate,7.9,6.9,7.6,7.7,7.5,8.5,7.0
2,a-place-to-dream-zippori,9.9,9.3,9.7,9.8,9.5,9.5,7.5
3,alaska-inn,8.4,6.9,7.2,7.3,7.6,9.0,8.6


In [45]:
df.style.set_properties(**{'text-align': 'right'}).set_table_styles([ dict(selector='th', props=[('text-align', 'right')] ) ])
#dfStyler = df.style.set_properties(**{'text-align': 'right'})
#dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'right')])])
display(df)

,Hotel_Names,Staff_Rating,Facilities_Rating,Cleanliness_Rating,Comfort_Rating,Value_For_Money_Rating,Location_Rating,Free_WiFi_Rating
0,kibbutz-hatzor-a-room-close-to-ashdod,9.4,9.0,8.9,9.2,9.0,9.1,9.1
1,khvzt-nylbyts-39,8.4,7.8,7.8,7.7,7.2,8.1,4.6
2,xxxx-xc-x-xx,9.3,8.0,8.7,8.4,8.4,9.0,8.0
3,arava-hostel,8.3,7.6,7.6,8.1,7.4,8.3,7.3
4,stlbt-bmdbr,9.8,8.8,9.2,8.9,9.3,8.7,9.6
5,motel-aviv,9.5,9.1,9.6,9.3,9.0,8.8,10
6,sapir,9.5,8.8,9.3,9.2,9.3,9.0,10
7,astral-seaside,9.8,9.6,9.6,9.6,9.4,9.3,9.4
8,palms-hotel-eilat,9.4,9.3,9.7,9.6,9.5,9.5,8.8
9,desert-iris,9.8,8.6,9.2,8.9,9.1,9.2,7.5
